In [1]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

当前运行环境缺少部分需要的环境变量，可能导致项目运行报错，请确认以下环境变量已经配置并赋值: BK_PAAS2_URL
注意，当前版本为社区版环境，请确认 BKPAAS_MAJOR_VERSION 已经配置并正确赋值


In [2]:
# 用于生成查询参数
from fta_web.models.alert import MetricRecommendationFeedback
from django.db.models import Q, Count

params = MetricRecommendationFeedback.objects.all().values_list('alert_metric_id', 'recommendation_metric_hash',
                                                                'bk_biz_id', 'create_user').annotate(count=Count(1))[
         :100]
# print(list(resut))
alert_metric_ids = []
rec_metric_hashs = []
bk_biz_ids = []
usernames = []

for ietm in params:
    # print(ietm)
    alert_metric_ids.append(ietm[0])
    rec_metric_hashs.append(ietm[1])
    bk_biz_ids.append(ietm[2])
    usernames.append(ietm[3])


alert_metric_ids=tuple(alert_metric_ids)
rec_metric_hashs=tuple(rec_metric_hashs)
bk_biz_ids=tuple(bk_biz_ids)
usernames=tuple(usernames)

#{'alert_metric_id': 'metric_0', 'recommendation_metric_hash': 'metric_hash_0', 'bk_biz_id': 1, 'count': 10}


In [3]:
# 循环查询数据库，并保存结果
from fta_web.alert.resources import MetricRecommendationFeedbackResource
import json
import time


def get_feedback_count(alert_metric_id, recommendation_metric_hash, bk_biz_id):
    """获取业务下，告警指标,被推荐指标关系下的点赞和点踩数

    :param alert_metric_id: 告警指标名
    :param recommendation_metric: 被推荐指标
    :param bk_biz_id: 业务id
    :return: (点赞数,点踩数)
    """
    feedback_annotate = (
        MetricRecommendationFeedback.objects.filter(
            alert_metric_id=alert_metric_id,
            recommendation_metric_hash=recommendation_metric_hash,
            bk_biz_id=bk_biz_id,
        )
        .values("feedback")
        .annotate(Count("feedback"))
    )

    good_count = 0
    bad_count = 0
    for feedback_annotate_item in feedback_annotate:
        if feedback_annotate_item["feedback"] == MetricRecommendationFeedback.FeedBackChoices.GOOD:
            good_count = feedback_annotate_item["feedback__count"]
        elif feedback_annotate_item["feedback"] == MetricRecommendationFeedback.FeedBackChoices.BAD:
            bad_count = feedback_annotate_item["feedback__count"]
    return good_count, bad_count


def loop_query():
    result = {}
    for alert_metric_id, recommendation_metric_hash, bk_biz_id, _, _ in params:
        good_count, bad_count = get_feedback_count(alert_metric_id, recommendation_metric_hash, bk_biz_id)
        result[(alert_metric_id, recommendation_metric_hash, bk_biz_id)] = [good_count, bad_count]

    print(len(result))
    return result

# loop_query()


In [4]:
# 批量查询获取数据
from typing import List, Dict
from collections import defaultdict
from functools import reduce
import operator


def loop_query():
    """
    获取业务下，告警指标,被推荐指标关系下的点赞和点踩数
    """
    result = {}
    for alert_metric_id, recommendation_metric_hash, bk_biz_id, _, _ in params:
        good_count, bad_count = get_feedback_count(alert_metric_id, recommendation_metric_hash, bk_biz_id)
        result[(alert_metric_id, recommendation_metric_hash, bk_biz_id)] = [good_count, bad_count]

    print("length:", len(result))
    return result


def batch_query():
    """
    批量获取业务下，告警指标,被推荐指标关系下的点赞和点踩数
    """
    get_count_result = MetricRecommendationFeedbackResource.get_feedback_count_batch(alert_metric_ids, rec_metric_hashs,
                                                                                     bk_biz_ids)
    print("length:", len(get_count_result))
    return get_count_result


loog_query_result = loop_query()
batch_query_result = loop_query()
print("两查询结果是否一致：", loog_query_result == batch_query_result)
print(batch_query_result)
print(loog_query_result)


# batch_query()

length: 4
length: 4
两查询结果是否一致： True
{('metric_0', 'metric_hash_0', 0): [17, 12], ('metric_0', 'metric_hash_0', 1): [12, 14], ('metric_0', 'metric_hash_0', 2): [18, 12], ('metric_0', 'metric_hash_0', 3): [18, 19]}
{('metric_0', 'metric_hash_0', 0): [17, 12], ('metric_0', 'metric_hash_0', 1): [12, 14], ('metric_0', 'metric_hash_0', 2): [18, 12], ('metric_0', 'metric_hash_0', 3): [18, 19]}


In [5]:
def get_feedback(alert_metric_id, recommendation_metric, bk_biz_id, username):
    """获取用户的反馈

    :param alert_metric_id: 告警指标名
    :param recommendation_metric: 被推荐指标
    :param bk_biz_id: 业务id
    :param username: 用户名
    :return: 业务下对应告警指标和被推荐指标的点赞数和点踩数，以及用户的反馈
    """
    good_count, bad_count = get_feedback_count(alert_metric_id, recommendation_metric, bk_biz_id)

    username_feedback = MetricRecommendationFeedback.objects.filter(
        alert_metric_id=alert_metric_id,
        recommendation_metric_hash=recommendation_metric,
        bk_biz_id=bk_biz_id,
        create_user=username,
    ).first()
    return {
        "good": good_count,
        "bad": bad_count,
        "self": username_feedback.feedback if username_feedback else None,
    }




In [6]:
def query_feedback_loop():
    """
    循环获取用户的反馈
    """
    result = {}
    for alert_id, rec_metric_hash, bk_biz_id, username, _ in params:
        result[(alert_id, rec_metric_hash, bk_biz_id, username)] = get_feedback(alert_id, rec_metric_hash, bk_biz_id,
                                                                                username)
    for item in result.items():
        print(item)
    return result


def query_feedback_batch():
    """
    批量获取用户的反馈
    """
    res = MetricRecommendationFeedbackResource.get_feedback_batch(alert_metric_ids, rec_metric_hashs, bk_biz_ids,
                                                                  usernames)

    # res = get_feedback_batch(alert_metric_ids, rec_metric_hashs, bk_biz_ids, usernames)
    for item in res.items():
        print(item)
    return res


feedback_loop_result = query_feedback_loop()
print()
feedback_batch_result = query_feedback_batch()
print(feedback_loop_result == feedback_batch_result)



(('metric_0', 'metric_hash_0', 0, 'user_1035'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_0', 0, 'user_1276'), {'good': 17, 'bad': 12, 'self': 'good'})
(('metric_0', 'metric_hash_0', 0, 'user_2605'), {'good': 17, 'bad': 12, 'self': 'good'})
(('metric_0', 'metric_hash_0', 0, 'user_2872'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_0', 0, 'user_2928'), {'good': 17, 'bad': 12, 'self': 'good'})
(('metric_0', 'metric_hash_0', 0, 'user_3459'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_0', 0, 'user_3589'), {'good': 17, 'bad': 12, 'self': 'good'})
(('metric_0', 'metric_hash_0', 0, 'user_3734'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_0', 0, 'user_4263'), {'good': 17, 'bad': 12, 'self': 'good'})
(('metric_0', 'metric_hash_0', 0, 'user_4676'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_0', 0, 'user_4912'), {'good': 17, 'bad': 12, 'self': 'bad'})
(('metric_0', 'metric_hash_

In [7]:
from fta_web.alert.resources import MetricRecommendationResource


def perform_request_batch(self, validated_request_data,username_set):
    # result = super(MetricRecommendationResource, self).perform_request(validated_request_data)

    """
    alert_metric_ids ['metric_0', 'metric_0']
    rec_metric_hashs ['metric_hash_0', 'metric_hash_0']
    bk_biz_ids [0, 0]
    usernames ['user_1035', 'user_1276']
    """
    result=validated_request_data

    # username = get_request_username()
    # 参数列表,每个列表同位置的元素一一对应，共同组成一对查询参数
    alert_metric_ids = []
    rec_metric_hashs = []
    bk_biz_ids = []
    usernames = []

    # 收集需要查询的参数
    query_params = {}
    for label_info in result.get("recommended_metrics", []):
        for metric_info in label_info["metrics"]:
            for recommend_panel in metric_info["panels"]:
                recommend_info = recommend_panel["recommend_info"]
                alert_metric_id = recommend_info["src_metric_id"]
                recommendation_metric = recommend_panel["id"]
                bk_biz_id = recommend_panel["bk_biz_id"]

                # 将参数放入列表
                alert_metric_ids.append(alert_metric_id)
                # rec_metric_hashs.append(
                #     MetricRecommendationFeedback.generate_recommendation_metric_hash(recommendation_metric))
                rec_metric_hashs.append(recommendation_metric)
                bk_biz_ids.append(bk_biz_id)
                username = next(username_set)
                usernames.append(username)

                # 收集查询参数,并保留对应的recommend_panel
                query_params[(alert_metric_id, recommendation_metric, bk_biz_id, username)] = recommend_panel

    # 批量查询反馈信息
    feedback_results = MetricRecommendationFeedbackResource.get_feedback_batch(alert_metric_ids, rec_metric_hashs,
                                                                               bk_biz_ids, usernames)

    # print(feedback_results)

    for (alert_metric_id, recommendation_metric, bk_biz_id, username), recommend_panel in query_params.items():
        recommend_panel["feedback"] = feedback_results[(alert_metric_id, recommendation_metric, bk_biz_id, username)]

    return result




In [8]:
import copy

def perform_request_loop(self, validated_request_data, username_set):
    # result = super(MetricRecommendationResource, self).perform_request(validated_request_data)
    """
    alert_metric_ids ['metric_0', 'metric_0']
    rec_metric_hashs ['metric_hash_0', 'metric_hash_0']
    bk_biz_ids [0, 0]
    usernames ['user_1035', 'user_1276']
    """
    result = validated_request_data

    for label_info in result.get("recommended_metrics", []):
        for metric_info in label_info["metrics"]:
            for recommend_panel in metric_info["panels"]:
                recommend_info = recommend_panel["recommend_info"]
                alert_metric_id = recommend_info["src_metric_id"]
                recommendation_metric = recommend_panel["id"]
                bk_biz_id = recommend_panel["bk_biz_id"]
                username = next(username_set)
                feedback = get_feedback(alert_metric_id, recommendation_metric, bk_biz_id, username)
                recommend_panel["feedback"] = feedback

    return result


request_data = {
    "recommended_metrics": [{
        "metrics": [{
            "panels": [
                {"recommend_info": {"src_metric_id": "metric_0"},
                 "id": 'metric_hash_0',
                 "bk_biz_id": 0,
                 "feedback": {},
                 }
            ]
        }, {
            "panels": [
                {"recommend_info": {"src_metric_id": "metric_0"},
                 "id": 'metric_hash_0',
                 "bk_biz_id": 0,
                 "feedback": {},
                 }
            ]
        }]
    }]
}

username_set = iter(["user_1035", "user_1276"])

if __name__ == '__main__':
    # perform_request 方法测试
    instance = MetricRecommendationResource()
    
    # 循环查询，获取推荐指标信息
    result_loop = perform_request_loop(instance, request_data, copy.copy(username_set))
    # 批量查询，获取推荐指标信息
    result_batch = perform_request_batch(instance, request_data,copy.copy(username_set))
    print("量查询结果是否一致：",result_loop == result_batch)
    print(result_loop)
    print()
    print(result_batch)
    

量查询结果是否一致： True
{'recommended_metrics': [{'metrics': [{'panels': [{'recommend_info': {'src_metric_id': 'metric_0'}, 'id': 'metric_hash_0', 'bk_biz_id': 0, 'feedback': {'good': 17, 'bad': 12, 'self': 'bad'}}]}, {'panels': [{'recommend_info': {'src_metric_id': 'metric_0'}, 'id': 'metric_hash_0', 'bk_biz_id': 0, 'feedback': {'good': 17, 'bad': 12, 'self': 'good'}}]}]}]}

{'recommended_metrics': [{'metrics': [{'panels': [{'recommend_info': {'src_metric_id': 'metric_0'}, 'id': 'metric_hash_0', 'bk_biz_id': 0, 'feedback': {'good': 17, 'bad': 12, 'self': 'bad'}}]}, {'panels': [{'recommend_info': {'src_metric_id': 'metric_0'}, 'id': 'metric_hash_0', 'bk_biz_id': 0, 'feedback': {'good': 17, 'bad': 12, 'self': 'good'}}]}]}]}


In [9]:
# 断点测试
from fta_web.alert.resources import MetricRecommendationResource

params = {
    "bk_biz_id": 2,
    "alert_metric_id": 2,
    "recommendation_metric_id": 2
}

MetricRecommendationResource().perform_request(params)


KeyError: 'alert_id'